# QC of cellranger-arc mapping using Muon E-MTAB12919_E-MTAB12916

    Developed by: Christian Eger 
    Würzburg Institute for Systems Immunology - Faculty of Medicine - Julius Maximilian Universität Würzburg
    Created on: 240625
    Last modified: 240625

## Environment Prep

### import modules

In [10]:
import muon as mu
import os
import subprocess

### set default paths

In [14]:
data_path = os.path.join(os.getcwd(), '.data')
input_path = os.path.join(data_path, 'input')
output_path = os.path.join(data_path, 'output')

os.listdir(input_path)

['E-MTAB12916_E-MTAB12919.h5mu']

### helper functions

## QC

## Read h5mu into mdata object

In [12]:
mdata_input_path = os.path.join(input_path, 'E-MTAB12916_E-MTAB12919.h5mu')
mdata = mu.read_h5mu(mdata_input_path)
mdata

/home/ceger/miniforge3/envs/heart_mapping/lib/python3.10/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/home/ceger/miniforge3/envs/heart_mapping/lib/python3.10/site-packages/anndata/_core/anndata.py:430: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


MuData object with n_obs × n_vars = 298902 × 3775615
  2 modalities
    rna:	298902 x 70711
      obs:	'sample'
    atac:	298902 x 3704904
      obs:	'sample'

## Save mdata as h5mu

In [15]:
mdata_output_path = os.path.join(output_path, 'E-MTAB12916_E-MTAB12919-QC.h5mu')
mdata.write_h5mu(mdata_output_path)

os.listdir(output_path)

['E-MTAB12916_E-MTAB12919-QC.h5mu']